In [1]:
import sys, os, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [2]:
sys.path.append(
    "/Users/wiegerscheurer/repos/physicspred"
)  # To enable importing from repository folders

In [3]:
150/2.2
aars = np.array([.45, .45, .625], dtype=float)
# shuffle aars
np.random.shuffle(aars)

aars

array([0.625, 0.45 , 0.45 ])

In [4]:
from functions.physics import _rotate_90, _dir_to_vec, _vec_to_dir, _flip_dir
from functions.utilities import determine_sequence
from functions.analysis import (get_precision, 
                                get_data, 
                                get_false_negatives,
                                get_false_positives,
                                get_true_positives,
                                get_true_negatives,
                                filter_condition,
                                get_f1_score, 
                                get_rt, 
                                get_accuracy,
                                get_hit_rate,)


### BALANCING THEM TRIALS

In [ ]:
interactor_trial_options = ["45_top_r", "45_top_u",
                            "45_bottom_l", "45_bottom_d",
                            "135_top_l", "135_top_u",
                            "135_bottom_r", "135_bottom_d"]
empty_trial_options = ["none_l", "none_r", "none_u", "none_d"]  # For the none trials


bounce_options = [True, False]
rand_bounce_direction_options = ["left", "right"] * 2
random.shuffle(rand_bounce_direction_options)


target_baserate = .5
ball_change_options = [True] * 1 + [False] * int((1 / target_baserate) - 1)
# rand_speed_change_options = ["slower", "faster"]
natural_speed_variance = .5 #config["natural_speed_variance"]
avg_ball_speed = 11

ball_speed_options = list(
    np.arange(
        avg_ball_speed - natural_speed_variance,
        avg_ball_speed + (2 * natural_speed_variance),
        natural_speed_variance,
    )
)



trial_types = determine_sequence(n_trials, [1, 0], randomised=True) # 1 is interactor, 0 is empty trial

interactor_trials = balance_over_bool(trial_types, interactor_trial_options, randomised=True)

# Create deterministically randomised; balanced parameter sequences
trials = determine_sequence(n_trials, trial_options, randomised=True)

bounces = determine_sequence(n_trials, bounce_options, randomised=True)

rand_bounce_directions = get_phantbounce_sequence(trials, rand_bounce_direction_options, randomised=True) # Random phantom bounce direction

ball_changes = determine_sequence(n_trials, ball_change_options, randomised=True)
rand_speed_changes = determine_sequence(
    n_trials, rand_speed_change_options, randomised=True
)
ball_speeds = determine_sequence(n_trials, ball_speed_options, randomised=True)

ball_spawn_spread = config[
    "ball_spawn_spread"
]  # Margin around fixation where the ball can spawn (smaller = )


In [15]:
ui = "drek_spek_1"
ui[:-2]

'drek_spek'

In [29]:

from functions.utilities import check_balance, create_balanced_trial_design



def build_design_matrix(n_trials:int, verbose:bool=False):
    """
    Build a design matrix for a given number of trials.

    Parameters:
    - n_trials (int): The total number of trials.
    - verbose (bool): Whether to print verbose output.

    Returns:
    - design_matrix (pd.DataFrame): The resulting design matrix.
    """
    trials_per_fullmx = 192

    full_matrices = n_trials // trials_per_fullmx
    remainder = n_trials % trials_per_fullmx
    
    if verbose:
        print(f"Design matrix for {n_trials} trials, constituting {full_matrices} fully balanced matrices and {remainder} trials balanced approximately optimal.")
    
    if remainder > 0:
        initial_dm = create_balanced_trial_design(remainder)
    else:
        initial_dm = pd.DataFrame()
    
    for full_matrix in range(full_matrices + 1):
        dm = create_balanced_trial_design(192)
        if full_matrix == 0:
            design_matrix = initial_dm
        else:
            design_matrix = pd.concat([design_matrix, dm])
            
    # Shuffle the rows and reset the index
    design_matrix = design_matrix.sample(frac=1).reset_index(drop=True)
    return design_matrix


design_matrix = build_design_matrix(38, True)

check_balance(design_matrix)
design_matrix

Design matrix for 38 trials, constituting 0 fully balanced matrices and 38 trials balanced approximately optimal.
Creating balanced trial design with 38 trials
Creating balanced trial design with 192 trials
Total trials: 38

Trial type balance:
trial_type
empty         19
interactor    19
Name: count, dtype: int64

Trial option balance for interactor trials:
trial_option
135_bottom_d    3
135_bottom_r    2
135_top_l       3
135_top_u       2
45_bottom_d     2
45_bottom_l     2
45_top_r        2
45_top_u        3
Name: count, dtype: int64
Variance: 0.27

Trial option balance for empty trials:
trial_option
none_d    5
none_l    5
none_r    5
none_u    4
Name: count, dtype: int64
Variance: 0.25

Bounce balance:
bounce
True     20
False    18
Name: count, dtype: int64

Ball change balance:
ball_change
False    19
True     19
Name: count, dtype: int64

Ball speed balance:
ball_speed
6.00    20
6.25     6
6.50    12
Name: count, dtype: int64

Cross-tabulation of trial_type × bounce:
bounce  

,trial_type,trial_option,bounce,phant_bounce_direction,ball_change,ball_speed
0,empty,none_d,False,None,False,6.00
1,interactor,45_bottom_l,True,None,True,6.00
2,interactor,45_top_r,True,None,False,6.00
3,interactor,45_top_u,True,None,True,6.00
4,interactor,45_bottom_d,False,None,True,6.50
5,empty,none_r,True,right,True,6.50
6,empty,none_u,True,right,True,6.50
7,empty,none_l,False,None,False,6.00
8,empty,none_u,True,right,False,6.50
9,interactor,135_top_l,True,None,False,6.00


In [15]:
1000//192

5*192

1000%192

100//192

0

In [186]:
from functions.utilities import check_balance, create_balanced_trial_design
# load in csv file as pd.dataframe
df = pd.read_csv('/Users/wiegerscheurer/repos/physicspred/data/sub-224750/ball_hue/design_matrix.csv')

check_balance(df)

Total trials: 144

Trial type balance:
trial_type
interactor    96
empty         48
Name: count, dtype: int64

Trial option balance for interactor trials:
trial_option
135_bottom_d    12
135_bottom_r    12
135_top_l       12
135_top_u       12
45_bottom_d     12
45_bottom_l     12
45_top_r        12
45_top_u        12
Name: count, dtype: int64
Variance: 0.00

Trial option balance for empty trials:
trial_option
none_d    12
none_l    12
none_r    12
none_u    12
Name: count, dtype: int64
Variance: 0.00

Bounce balance:
bounce
True     72
False    72
Name: count, dtype: int64

Ball change balance:
ball_change
True     72
False    72
Name: count, dtype: int64

Ball speed balance:
ball_speed
6.00    48
6.25    48
6.50    48
Name: count, dtype: int64

Cross-tabulation of trial_type × bounce:
bounce      False  True 
trial_type              
empty          24     24
interactor     48     48

Cross-tabulation of trial_type × ball_change:
ball_change  False  True 
trial_type               
emp

#### Acquire data per sub

In [50]:
all_sub_names = ["wolpert", "wunger","scheur", "hendrik", "willem", "ikzelf", "melvin", "paulo", "yifan", "ann", "qifei", "mingyao", "bilge", "yanni", "eva"]
sub_stack = pd.DataFrame()

for sub_idx, sub in enumerate(all_sub_names):
    this_sub = get_data(subject=f"sub-{sub}", task="ball_hue")
    sub_stack = pd.concat([sub_stack, this_sub])
    
sub_stack.reset_index(inplace=True, drop=True)
# sub_stack

In [135]:
this_sub = random.choice(all_sub_names)
print(this_sub)
sub = get_data(subject=f"sub-{this_sub}", task="ball_hue")

simc_xpolc = get_rt(df=sub, sim_con=True, expol_con=True, return_df=True)["rt"]
simc_xpoli = get_rt(df=sub, sim_con=True, expol_con=False, return_df=True)["rt"]
simi_xpolc = get_rt(df=sub, sim_con=False, expol_con=True, return_df=True)["rt"]
simi_xpoli = get_rt(df=sub, sim_con=False, expol_con=False, return_df=True)["rt"]


all_kinds = [simc_xpolc, simc_xpoli, simi_xpolc, simi_xpoli]

for kind in all_kinds:
    print(len(kind))
    print(kind.mean(), kind.std())


bilge


FileNotFoundError: [Errno 2] No such file or directory: '/Users/wiegerscheurer/repos/physicspred/data/sub-bilge/ball_hue/'

#### Integrate in pilot script

In [187]:
sim_cons = [True, False]
expol_cons = [True, False]
# randsub = random.choice(all_sub_names)
randsub = "224750"
print(f"Now looking at : {randsub}")
for sim_con in sim_cons:
    
    for expol_con in expol_cons:
        
        print(f"sim_con: {sim_con}, expol_con: {expol_con}")
        dat = get_data(subject=f"sub-{randsub}", task="ball_hue")
        dat_filt = dat[(dat["sim_congruent"] == sim_con) & (dat["abs_congruent"] == expol_con)]
        print(len(dat_filt))

Now looking at : 224750
sim_con: True, expol_con: True
24
sim_con: True, expol_con: False
48
sim_con: False, expol_con: True
48
sim_con: False, expol_con: False
24


In [192]:
wie = get_data(subject="sub-224750", task="ball_hue")

n_targets = len(wie[wie["ball_change"] == 1]) # 20
n_nones = len(wie[wie["interactor"] == "none_d"])# 20
n_nones

# sort dataframe based on trials column
wie.sort_values(by="interactor", inplace=True)

wie["response"]

15      NaN
100    left
102     NaN
43     left
48      NaN
       ... 
115      up
124      up
116    left
62      NaN
70     left
Name: response, Length: 144, dtype: object

In [193]:
wie

,trial_type,trial,ball_speed,interactor,bounce,bounce_moment,random_bounce_direction,target_onset,ball_change,target_color,...,start_pos,end_pos,abs_rfup,abs_rfright,abs_rfdown,abs_rfleft,sim_rfup,sim_rfright,sim_rfdown,sim_rfleft
15,interactor,16,6.00,135_bottom_d,True,3.980140,NaN,NaN,False,NaN,...,up,left,"(0, 0)","(0, 0)","(1, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)"
100,interactor,101,6.00,135_bottom_d,True,4.166790,NaN,4.432431,True,[0.45 0.45 0.695],...,up,left,"(0, 0)","(0, 0)","(1, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)"
102,interactor,103,6.25,135_bottom_d,True,3.950836,NaN,NaN,False,NaN,...,up,left,"(0, 0)","(0, 0)","(1, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)"
43,interactor,44,6.25,135_bottom_d,True,3.949038,NaN,4.217447,True,[0.45 0.45 0.695],...,up,left,"(0, 0)","(0, 0)","(1, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)"
48,interactor,49,6.00,135_bottom_d,False,NaN,NaN,NaN,False,NaN,...,up,down,"(0, 0)","(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)","(0, 1)","(1, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,empty,116,6.00,none_u,False,NaN,NaN,NaN,False,NaN,...,down,up,"(1, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)"
124,empty,125,6.50,none_u,False,NaN,NaN,4.225236,True,[0.45 0.45 0.695],...,down,up,"(1, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)"
116,empty,117,6.25,none_u,True,3.942658,left,4.208010,True,[0.45 0.45 0.695],...,down,left,"(1, 0)","(0, 0)","(0, 0)","(0, 1)","(1, 0)","(0, 0)","(0, 0)","(0, 1)"
62,empty,63,6.25,none_u,True,3.951371,left,NaN,False,NaN,...,down,left,"(1, 0)","(0, 0)","(0, 0)","(0, 1)","(1, 0)","(0, 0)","(0, 0)","(0, 1)"


In [194]:

hit_rate = get_hit_rate(wie, False, False)
print(hit_rate)
wie_filt = filter_condition(wie, False, False)

fp = get_false_positives(wie_filt)
fn = get_false_negatives(wie_filt)
tp = get_true_positives(wie_filt)
tn = get_true_negatives(wie_filt)


print(f"False positives: {fp}")
print(f"False negatives: {fn}")
print(f"True positives: {tp}")
print(f"True negatives: {tn}")


hr = (tp) / (tp + fn)

hr
# get_misses(filter_condition(wie, False, False))

0.9166666666666666
False positives: 4
False negatives: 1
True positives: 11
True negatives: 8


0.9166666666666666

In [195]:
for sim_con in [True, False]:
    for xpol_con in [True, False]:
        print(f"sim_con: {sim_con}, xpol_con: {xpol_con}")
        print(f"{get_hit_rate(wie, sim_con, xpol_con):.2f}\n")

sim_con: True, xpol_con: True
1.00

sim_con: True, xpol_con: False
1.00

sim_con: False, xpol_con: True
1.00

sim_con: False, xpol_con: False
0.92



In [196]:
cc = filter_condition(wie, True, True)
ci = filter_condition(wie, True, False)
ic = filter_condition(wie, False, True)
ii = filter_condition(wie, False, False)

cc_targets = cc[cc["ball_change"] == 1]
ci_targets = ci[ci["ball_change"] == 1]
ic_targets = ic[ic["ball_change"] == 1]
ii_targets = ii[ii["ball_change"] == 1]

print(f"cc targets: {len(cc_targets)}")
print(f"ci targets: {len(ci_targets)}")
print(f"ic targets: {len(ic_targets)}")
print(f"ii targets: {len(ii_targets)}")

print(f"cc: {len(cc)}")
print(f"ci: {len(ci)}")
print(f"ic: {len(ic)}")
print(f"ii: {len(ii)}")


cc targets: 12
ci targets: 24
ic targets: 24
ii targets: 12
cc: 24
ci: 48
ic: 48
ii: 24


In [203]:
allen = get_data(subject=None, task="ball_hue")
# allen = get_data(subject="sub-224750", task="ball_hue")

In [204]:
# for this_name in all_sub_names:
    # wie = get_data(subject=f"sub-{this_name}", task="ball_hue")
wie = allen


# Get hit rates for each condition
simc_xpolc = get_hit_rate(df=wie, sim_con=True, expol_con=True, return_df=True)["accuracy"]
simc_xpoli = get_hit_rate(df=wie, sim_con=True, expol_con=False, return_df=True)["accuracy"]
simi_xpolc = get_hit_rate(df=wie, sim_con=False, expol_con=True, return_df=True)["accuracy"]
simi_xpoli = get_hit_rate(df=wie, sim_con=False, expol_con=False, return_df=True)["accuracy"]
# simc_xpolc = get_precision(df=wie, sim_con=True, expol_con=True, return_df=True)["accuracy"] # WHy doesn't this work?
# simc_xpoli = get_precision(df=wie, sim_con=True, expol_con=False, return_df=True)["accuracy"]
# simi_xpolc = get_precision(df=wie, sim_con=False, expol_con=True, return_df=True)["accuracy"]
# simi_xpoli = get_precision(df=wie, sim_con=False, expol_con=False, return_df=True)["accuracy"]

cc_rt = get_rt(df=wie, sim_con=True, expol_con=True, return_df=False)
ci_rt = get_rt(df=wie, sim_con=True, expol_con=False, return_df=False)
ic_rt = get_rt(df=wie, sim_con=False, expol_con=True, return_df=False)
ii_rt = get_rt(df=wie, sim_con=False, expol_con=False, return_df=False)

print(f"Subject missed {get_false_negatives(wie)} trials")
# print(f"simc_xpolc: {simc_xpolc.mean()}, simc_xpoli: {simc_xpoli.mean()}, simi_xpolc: {simi_xpolc.mean()}, simi_xpoli: {simi_xpoli.mean()}")

# Create data for the 2x2 heatmap
z_values = [[simc_xpolc.mean(), simc_xpoli.mean()], 
            [simi_xpolc.mean(), simi_xpoli.mean()]]

# Create a figure with subplot for heatmap
fig = go.Figure()

# Add heatmap
fig.add_trace(go.Heatmap(
    z=z_values,
    x=['Congruent', 'Incongruent'],
    y=['Congruent', 'Incongruent'],
    text=[[f"{simc_xpolc.mean():.3f}<br>#trials: {len(simc_xpolc)}<br>rt: {cc_rt:.2f}s", f"{simc_xpoli.mean():.3f}<br>#trials: {len(simc_xpoli)}<br>rt: {ci_rt:.2f}s"], 
        [f"{simi_xpolc.mean():.3f}<br>#trials: {len(simi_xpolc)}<br>rt: {ic_rt:.2f}s", f"{simi_xpoli.mean():.3f}<br>#trials: {len(simi_xpoli)}<br>rt: {ii_rt:.2f}s"]],
    texttemplate="%{text}",
    textfont={"size":14},
    colorscale="magma_r",
    showscale=True,
    colorbar=dict(title="Hit Rate"),
    zmin=0,  # Set minimum value for the color scale
    zmax=1   # Set maximum value for the color scale
))

# Update layout
fig.update_layout(
    title=f"Hit Rate by Condition",
    xaxis=dict(title="Motion Extrapolation", side="bottom"),
    yaxis=dict(title="Simulation"),
    width=600,
    height=500
)

# Show the figure
fig.show()

Subject missed 9 trials


In [205]:

# Create an empty list to store all data frames
all_data = []

# Define types and their colors
types = ['Simcon + Xpolcon', 'Simcon + Xpolinc', 'Siminc + Xpolcon', 'Siminc + Xpolinc']

# Generate a colormap
cmap = plt.get_cmap('YlOrRd')  # You can choose a different colormap if you prefer
col_factor = 50
col_icept = 150
# Create the colors dictionary
type_colors = {name: f'rgba({int(cmap(i*col_factor + col_icept)[0]*255)}, {int(cmap(i*col_factor + col_icept)[1]*255)}, {int(cmap(i*col_factor + col_icept)[2]*255)}, 0.5)' 
          for i, name in enumerate(types)}

# Loop through all subjects (using the same structure as your original code)
# for sub_name in all_sub_names:
# for sub_name in ["139166"]:
for sub_name in ["allen"]:
    # sub = get_data(subject=f"{sub_name}", task="ball_hue")
    sub = allen
    simc_xpolc = get_rt(df=sub, sim_con=True, expol_con=True, return_df=True)["rt"]
    simc_xpoli = get_rt(df=sub, sim_con=True, expol_con=False, return_df=True)["rt"]
    simi_xpolc = get_rt(df=sub, sim_con=False, expol_con=True, return_df=True)["rt"]
    simi_xpoli = get_rt(df=sub, sim_con=False, expol_con=False, return_df=True)["rt"]
    
    # Create DataFrames without subject column - we'll aggregate across all subjects
    simc_xpolc_df = pd.DataFrame({
        'Reaction Time': simc_xpolc,
        'Type': 'Simcon + Xpolcon'
    })
    
    simc_xpoli_df = pd.DataFrame({
        'Reaction Time': simc_xpoli,
        'Type': 'Simcon + Xpolinc'
    })
    
    simi_xpolc_df = pd.DataFrame({
        'Reaction Time': simi_xpolc,
        'Type': 'Siminc + Xpolcon'
    })
    
    simi_xpoli_df = pd.DataFrame({
        'Reaction Time': simi_xpoli,
        'Type': 'Siminc + Xpolinc'
    })
    
    # Append all DataFrames to our list
    all_data.extend([simc_xpolc_df, simc_xpoli_df, simi_xpolc_df, simi_xpoli_df])

# Combine all the data into a single DataFrame
combined_data = pd.concat(all_data, ignore_index=True)

# Calculate mean reaction time for each type
type_means = combined_data.groupby('Type')['Reaction Time'].mean().to_dict()

# Create the plot
fig = go.Figure()

# Add violin plots for each type
for i, type_name in enumerate(types):
    subset = combined_data[combined_data['Type'] == type_name]
    
    fig.add_trace(go.Violin(
        x=[type_name] * len(subset),  # Use type name directly as x value
        y=subset['Reaction Time'],
        name=type_name,
        legendgroup=type_name,
        showlegend=True,
        box_visible=True,
        meanline_visible=True,
        points='all',
        jitter=0.2,
        pointpos=0.5,
        line_color=type_colors[type_name],
        side='negative',
        width=.5,
        spanmode='soft'
    ))

# Connect the means with a line
fig.add_trace(go.Scatter(
    x=types,
    y=[type_means[t] for t in types],
    mode='lines+markers',
    line=dict(color='rgba(0, 0, 0, 0.7)', width=8),
    marker=dict(
        size=14,  # Increased marker size
        color=[type_colors[t].replace('0.5', '1') for t in types],
        line=dict(color='black', width=4)
    ),
    name='Mean RT',
    hovertemplate='Mean: %{y:.3f}s<extra></extra>'
))

# Define a common font style for consistent text appearance
font_style = dict(
    family="Arial, sans-serif",
    size=20,  # Larger font size
    color="black"
)

# Update layout with enhanced text styling
fig.update_layout(
    title=dict(
        text="Average Reaction Times Across All Subjects\n\n",
        font=dict(size=25, family="Arial, sans-serif", color="black"),
        x=0.5,
        y=.985
    ),
    yaxis=dict(
        title=dict(
            text="Reaction Time (s)",
            font=font_style
        ),
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    xaxis=dict(
        title=dict(
            text="Response Type",
            font=font_style
        ),
        tickmode='array',
        tickvals=types,
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.01,
        xanchor="center",
        x=0.5,
        font=dict(size=18, family="Arial, sans-serif"),
        borderwidth=2,
        bordercolor="White",
        title=dict(
            text="Response Type",
            font=dict(size=18, family="Arial, sans-serif")
        )
    ),
    violinmode='overlay',
    # height=1200,
    # width=1200,
    height=1200,
    width=1200,
    font=font_style,  # Default font for all other text elements
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Add annotations for the mean values
for i, type_name in enumerate(types):
    fig.add_annotation(
        x=type_name,
        y=type_means[type_name],
        text=f"{type_means[type_name]:.3f}s",
        showarrow=True,
        arrowhead=2,
        arrowsize=1.5,  # Slightly larger arrowhead
        arrowwidth=3,    # Thicker arrow
        arrowcolor='black',
        ax=120,
        ay=-90,
        font=dict(size=25, color='black', family="Arial, sans-serif")
    )

# Make the axis lines thicker
fig.update_xaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=False)
fig.update_yaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()




In [34]:

# Create an empty list to store all data frames
all_data = []

# Define types and their colors
types = ['Abstraction', 'Simulation', 'Both']

# Generate a colormap
cmap = plt.get_cmap('YlOrRd')  # You can choose a different colormap if you prefer
col_factor = 50
col_icept = 150
# Create the colors dictionary
type_colors = {name: f'rgba({int(cmap(i*col_factor + col_icept)[0]*255)}, {int(cmap(i*col_factor + col_icept)[1]*255)}, {int(cmap(i*col_factor + col_icept)[2]*255)}, 0.5)' 
          for i, name in enumerate(types)}

# Loop through all subjects (using the same structure as your original code)
for sub_name in all_sub_names:
    sub = get_data(subject=f"sub-{sub_name}", task="ball_hue")
    # Get your data as before
    rt_dubs = get_rt(sub, hypothesis="both", include_dubtrials="only", return_df=True)
    rt_separate = get_rt(sub, hypothesis="both", include_dubtrials=False, return_df=True)
    sim_rt = rt_separate["simulation"]["rt"]
    abs_rt = rt_separate["abstraction"]["rt"]
    both_rt = rt_dubs["sim + abs"]["rt"]
    
    # Create DataFrames without subject column - we'll aggregate across all subjects
    sim_df = pd.DataFrame({
        'Reaction Time': sim_rt,
        'Type': 'Simulation'
    })
    
    abs_df = pd.DataFrame({
        'Reaction Time': abs_rt,
        'Type': 'Abstraction'
    })
    
    both_df = pd.DataFrame({
        'Reaction Time': both_rt,
        'Type': 'Both'
    })
    
    # Append all DataFrames to our list
    all_data.extend([abs_df, sim_df, both_df])

# Combine all the data into a single DataFrame
combined_data = pd.concat(all_data, ignore_index=True)

# Calculate mean reaction time for each type
type_means = combined_data.groupby('Type')['Reaction Time'].mean().to_dict()

# Create the plot
fig = go.Figure()

# Add violin plots for each type
for i, type_name in enumerate(types):
    subset = combined_data[combined_data['Type'] == type_name]
    
    fig.add_trace(go.Violin(
        x=[type_name] * len(subset),  # Use type name directly as x value
        y=subset['Reaction Time'],
        name=type_name,
        legendgroup=type_name,
        showlegend=True,
        box_visible=True,
        meanline_visible=True,
        points='all',
        jitter=0.2,
        pointpos=0.5,
        line_color=type_colors[type_name],
        side='negative',
        width=.5,
        spanmode='soft'
    ))

# Connect the means with a line
fig.add_trace(go.Scatter(
    x=types,
    y=[type_means[t] for t in types],
    mode='lines+markers',
    line=dict(color='rgba(0, 0, 0, 0.7)', width=8),
    marker=dict(
        size=14,  # Increased marker size
        color=[type_colors[t].replace('0.5', '1') for t in types],
        line=dict(color='black', width=4)
    ),
    name='Mean RT',
    hovertemplate='Mean: %{y:.3f}s<extra></extra>'
))

# Define a common font style for consistent text appearance
font_style = dict(
    family="Arial, sans-serif",
    size=20,  # Larger font size
    color="black"
)

# Update layout with enhanced text styling
fig.update_layout(
    title=dict(
        text="Average Reaction Times Across All Subjects\n\n",
        font=dict(size=25, family="Arial, sans-serif", color="black"),
        x=0.5,
        y=.985
    ),
    yaxis=dict(
        title=dict(
            text="Reaction Time (s)",
            font=font_style
        ),
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    xaxis=dict(
        title=dict(
            text="Response Type",
            font=font_style
        ),
        tickmode='array',
        tickvals=types,
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.01,
        xanchor="center",
        x=0.5,
        font=dict(size=18, family="Arial, sans-serif"),
        borderwidth=2,
        bordercolor="White",
        title=dict(
            text="Response Type",
            font=dict(size=18, family="Arial, sans-serif")
        )
    ),
    violinmode='overlay',
    # height=1200,
    # width=1200,
    height=800,
    width=800,
    font=font_style,  # Default font for all other text elements
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Add annotations for the mean values
for i, type_name in enumerate(types):
    fig.add_annotation(
        x=type_name,
        y=type_means[type_name],
        text=f"{type_means[type_name]:.3f}s",
        showarrow=True,
        arrowhead=2,
        arrowsize=1.5,  # Slightly larger arrowhead
        arrowwidth=3,    # Thicker arrow
        arrowcolor='black',
        ax=120,
        ay=-90,
        font=dict(size=25, color='black', family="Arial, sans-serif")
    )

# Make the axis lines thicker
fig.update_xaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=False)
fig.update_yaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()